In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt

torch.manual_seed(1)

EPOCH=1
BATCH_SIZE=50
LR=0.001
DOWNLOAD_MNIST=True

train_data=torchvision.datasets.MNIST(
    root='./mnist',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=DOWNLOAD_MNIST,
)

In [ ]:
test_data=torchvision.datasets.MNIST(root='./mnist', train=False)

train_loader=Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

# unsqueeze是为了将tensor从2000*28*28变成2000*1*28*28, 从而符合in_channels=1的CNN输入设置
test_x=torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000]/255
test_y=test_data.test_labels[:2000].numpy()

print(test_x.size())


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1=nn.Sequential(
            nn.Conv2d(
                in_channels=1, 
                out_channels=16,
                kernel_size=5, 
                stride=1,
                padding=2,
            ), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2=nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.out=nn.Linear(32*7*7, 10)
        
    def forward(self, x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=x.view(x.size(0), -1)
        output=self.out(x)
        
        return output

cnn=CNN()
print(cnn)

In [ ]:
optimizer=torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func=nn.CrossEntropyLoss()

for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):
        output=cnn(b_x)
        loss=loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step%50==0:
            output=cnn(test_x)
            pred_y=torch.max(output, 1)[1].data.numpy()
            accuracy=float((pred_y==test_y).astype(int).sum())/float(test_y.size)
            
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)
        

In [ ]:
test_output=cnn(test_x[:10])
pred_y=torch.max(test_output, 1)[1].numpy().squeeze()

print(pred_y, 'prediction number')
print(test_y[:10].numpy(), 'real_number')